In [164]:
import pandas as pd
import numpy as np
import os
import cv2

In [165]:
df = pd.read_csv('ADL-Rundle-6/det/det.txt', sep=',', header=None)

In [166]:
df.columns = ['frame', 'id', 'bb_left', 'bb_top', 'bb_width','bb_height', 'conf', 'x', 'y', 'z']

1. frame = frame number
2. id = number identifies that object as belonging to a trajectory by assigning a unique ID (set to −1 in a detection file, as no ID is assigned yet).
3. bb_left, bb_top, bb_width, bb_height: bounding box position in 2D image coordinates i.e. the top-left corner as well as width and height
4. conf: detection confidence score
5. x,y,z: the world coordinates are ignored for the 2D challenge and can be filled with -1

In [167]:
df.head(n=10)

,frame,id,bb_left,bb_top,bb_width,bb_height,conf,x,y,z
0,1,-1,1689,385,146.620,332.710,67.5670,-1,-1,-1
1,1,-1,1303,503,61.514,139.590,29.4390,-1,-1,-1
2,1,-1,1258,569,40.123,91.049,19.6010,-1,-1,-1
3,1,-1,31,525,113.370,257.270,17.0130,-1,-1,-1
4,1,-1,1800,483,94.660,214.810,11.9490,-1,-1,-1
5,2,-1,1689,385,146.620,332.710,66.7250,-1,-1,-1
6,2,-1,1312,503,61.514,139.590,36.6140,-1,-1,-1
7,2,-1,1744,476,123.420,280.060,16.9760,-1,-1,-1
8,2,-1,1254,537,52.000,118.000,15.9790,-1,-1,-1
9,2,-1,55,542,94.660,214.810,9.3326,-1,-1,-1


In [168]:
def compute_iou(boxA, boxB):
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])

    # Compute the area of intersection
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Compute the area of both bounding boxes
    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]

    # Compute the intersection over union
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [169]:
# Get unique frames
frames = df['frame'].unique()

# Initialize a dictionary to hold similarity matrices for each frame
similarity_matrices = {}

for frame in frames:
    # Filter detections for the current frame
    frame_detections = df[df['frame'] == frame]

    # Number of detections
    n_detections = len(frame_detections)

    # Initialize the similarity matrix for this frame
    sim_matrix = np.zeros((n_detections, n_detections))

    for i in range(n_detections):
        for j in range(i + 1, n_detections):
            # Compute IoU and update the similarity matrix
            boxA = frame_detections.iloc[i][['bb_left', 'bb_top', 'bb_width', 'bb_height']].values
            boxB = frame_detections.iloc[j][['bb_left', 'bb_top', 'bb_width', 'bb_height']].values
            sim_matrix[i, j] = compute_iou(boxA, boxB)
            sim_matrix[j, i] = sim_matrix[i, j]  # IoU is symmetric

    # Store the similarity matrix
    similarity_matrices[frame] = sim_matrix


In [170]:
similarity_matrices

{1: array([[0.        , 0.        , 0.        , 0.        , 0.12448738],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.12448738, 0.        , 0.        , 0.        , 0.        ]]),
 2: array([[0.        , 0.        , 0.36184537, 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.36184537, 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ]]),
 3: array([[0.        , 0.24498232, 0.        , 0.35728788, 0.        ,
         0.        , 0

In [171]:
# Placeholder sigma_iou threshold
sigma_iou = 0.4

# Placeholder for tracks - this will be a list of dicts with each dict representing a track
tracks = []

# Step 3: Associate detections to tracks
for frame_number in df['frame'].unique():
    frame_detections = df[df['frame'] == frame_number]
    
    # Compute IoU between each detection and each track
    for _, det in frame_detections.iterrows():
        det_box = [det['bb_left'], det['bb_top'], det['bb_left'] + det['bb_width'], det['bb_top'] + det['bb_height']]
        best_iou = 0
        best_track = None
        
        for track in tracks:
            last_det = track['detections'][-1]
            track_box = [last_det['bb_left'], last_det['bb_top'], last_det['bb_left'] + last_det['bb_width'], last_det['bb_top'] + last_det['bb_height']]
            iou = compute_iou(det_box, track_box)
            
            if iou > best_iou:
                best_iou = iou
                best_track = track
        
        # If the best IoU is above the threshold, update the track with the new detection
        if best_iou >= sigma_iou:
            best_track['detections'].append(det)
        else:
            # If no track is matched, create a new track
            tracks.append({'id': len(tracks) + 1, 'detections': [det]})
            
# Step 4: Track management
# Filter out the tracks that have no detections in the current frame
active_tracks = [track for track in tracks if track['detections'][-1]['frame'] == frame_number]
deleted_tracks = [track for track in tracks if track not in active_tracks]

In [172]:
images_directory =  'ADL-Rundle-6/img1'
for image_file in os.listdir(images_directory):
    if not image_file.endswith('.jpg'):
        continue 

    image_path = os.path.join(images_directory, image_file)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read {image_file}")
        continue

    frame_number = int(image_file.split('.')[0])  # Assumes filenames like '1.jpg', '2.jpg', etc.

    for track in tracks:
        current_id = track['id']
        for box_info in track['detections']:
            if box_info['frame'] == frame_number:
                # Draw a green bounding box on the image
                x, y, w, h = box_info['bb_left'], box_info['bb_top'], box_info['bb_width'], box_info['bb_height']
                x, y, w, h = int(x), int(y), int(w), int(h)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                # Put the ID on the image
                cv2.putText(image, f"ID: {current_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the modified image with boxes and IDs
    output_image_path = os.path.join('img2', f"{frame_number:06}.jpg")
    cv2.imwrite(output_image_path, image)

print("Images with bounding boxes and IDs have been saved.")

Images with bounding boxes and IDs have been saved.


In [173]:
# Directory where your JPEG frames are stored
frames_directory = 'img2'
frame_files = [f for f in os.listdir(frames_directory) if f.endswith('.jpg')]
frame_files.sort()

output_video_filename = 'output_video.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can choose other codecs like 'XVID' or 'MJPG'
frame_rate = 30  # Adjust the frame rate as needed
frame_size = (1920, 1080)  # Set the width and height of frames as needed
out = cv2.VideoWriter(output_video_filename, fourcc, frame_rate, frame_size)

# Loop through the frame files and add them to the video
for frame_file in frame_files:
    frame_path = os.path.join(frames_directory, frame_file)
    frame = cv2.imread(frame_path)

    # Check if the frame was read successfully
    if frame is None:
        print(f"Error: Could not read {frame_file}")
        continue

    # Write the frame to the video
    out.write(frame) 

# Release the VideoWriter object
out.release()

print(f"Video '{output_video_filename}' has been created.")


Video 'output_video.mp4' has been created.
